In [5]:
import pandas as pd
import numpy as np
import os

In [6]:
with open("data/us_train.text", "r") as sent_file:
    lines = sent_file.read().split("\n")

In [13]:
%time rows = [line.split(",") for line in lines if line]
%time rows = [row[:3] + [",".join(row[3:])] for row in rows]
# remove document start character 
rows[0][0] = rows[0][0][1:]

sentDf_cols = ['ItemID', 'Sentiment', 'SentimentSource', 'SentimentText']


%time sentDf_a = pd.DataFrame(rows[1:],columns=sentDf_cols)

#print(sentDf)

sentDf_a[["ItemID","Sentiment"]] = sentDf_a[["ItemID","Sentiment"]].astype(int)
%time sentDf_a["SentimentText"] = sentDf_a["SentimentText"].apply(lambda text: text.split())




CPU times: user 41 ms, sys: 14.8 ms, total: 55.8 ms
Wall time: 47.8 ms
CPU times: user 55 ms, sys: 17.3 ms, total: 72.3 ms
Wall time: 60.3 ms
CPU times: user 8.66 ms, sys: 1.53 ms, total: 10.2 ms
Wall time: 12 ms


ValueError: invalid literal for long() with base 10: 'glam on @user yesterday for #kcon makeup using @user in #featherette'

In [ ]:
path = os.getcwd()+"/data/"
with open(path+"us_train.text", "r") as tweet_text_file:
    lines = tweet_text_file.read().split("\n")
    

    
sentDf = pd.DataFrame(lines[:], columns=['SentimentText'])
%time sentDf["SentimentText"] = sentDf["SentimentText"].apply(lambda text: text.split())
#print(sentDf[1:])

In [14]:
import gensim
%time w2vM = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
dim = 300



CPU times: user 1min 23s, sys: 7.08 s, total: 1min 30s
Wall time: 1min 39s


In [10]:
import gensim


dim = 300
%time w2vM = gensim.models.Word2Vec(sentDf["SentimentText"], size=dim)


CPU times: user 2min 9s, sys: 1.85 s, total: 2min 11s
Wall time: 1min 5s


In [15]:
path = os.getcwd()+"/data/"
with open(path+"us_train.labels", "r") as labels_file:
    lines_test = labels_file.read().split("\n")
    
with open(path+"us_trial.labels", "r") as labels_file:
    lines_trial = labels_file.read().split("\n")
    
labels_train = pd.DataFrame(lines_test[:], columns=['SentimentLabels'])
labels_test = pd.DataFrame(lines_trial[:], columns=['SentimentLabels'])
#print(labels_test)

In [16]:
path = os.getcwd()+"/data/"
with open(path+"us_train.text", "r") as tweet_text_file:
    lines = tweet_text_file.read().split("\n")
    

    
sentDf_train = pd.DataFrame(lines[:], columns=['SentimentText'])
%time sentDf_train["SentimentText"] = sentDf_train["SentimentText"].apply(lambda text: text.split())
#print(sentDf[1:])
    
with open(path+"us_trial.text", "r") as tweet_text_file:
    lines = tweet_text_file.read().split("\n")

    
sentDf_test = pd.DataFrame(lines[:], columns=['SentimentText'])
%time sentDf_test["SentimentText"] = sentDf_test["SentimentText"].apply(lambda text: text.split())
#print(sentDf[1:])

CPU times: user 676 ms, sys: 106 ms, total: 782 ms
Wall time: 801 ms
CPU times: user 64.8 ms, sys: 6.67 ms, total: 71.4 ms
Wall time: 71.7 ms


In [17]:
# number of samples to aggregate
N_train = labels_train.size#int(1e5)
N_test = labels_test.size

In [18]:
%%time
# MEAN AGGREGATION
tvecs_train = np.array([np.array([w2vM[t] if t in w2vM
                                else np.zeros((dim,))
                            for t in twt]).mean(axis=0)
                 for twt in sentDf_train['SentimentText'][:N_train]])


tvecs_test = np.array([np.array([w2vM[t] if t in w2vM
                                else np.zeros((dim,))
                            for t in twt]).mean(axis=0)
                 for twt in sentDf_test['SentimentText'][:N_test]])


CPU times: user 36.2 s, sys: 2.34 s, total: 38.5 s
Wall time: 41 s


In [19]:

X_train = tvecs_train[:N_train]
X_test = tvecs_test[:N_test]

print(labels_train['SentimentLabels'][:N_train].values)
print(labels_test['SentimentLabels'][:N_test].values)

y_pre_train = labels_train['SentimentLabels'][:N_train].values
y_pre_test = labels_test['SentimentLabels'][:N_test].values

y_train = np.array(y_pre_train).astype(np.int)
y_test = np.array(y_pre_test).astype(np.int)

print(y_test)


['2' '17' '0' ..., '11' '19' '14']
['0' '7' '11' ..., '11' '0' '9']
[ 0  7 11 ..., 11  0  9]


In [15]:
# generate test/train split
ratio = 0.8
tidx = np.random.rand(N) < ratio
pidx = ~tidx

NameError: name 'N' is not defined

In [25]:
import sklearn
from sklearn import ensemble,svm,neural_network,discriminant_analysis
from sklearn.metrics import roc_curve,auc,precision_recall_curve

from matplotlib import pyplot as plt
% matplotlib inline

In [76]:
rf = sklearn.ensemble.RandomForestClassifier()
rf.max_depth = 5

%time rf.fit(X_test, y_test)

CPU times: user 4.08 s, sys: 80.7 ms, total: 4.16 s
Wall time: 4.32 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [26]:
num_nodes = 140
max_iters = 400
hidden_layer_config = (num_nodes,)

mlp = sklearn.neural_network.MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes= hidden_layer_config, learning_rate='constant',
       learning_rate_init=0.001, max_iter=max_iters, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)


#fit to training data
#%time mlp.fit(X[tidx],y_train[tidx])
%time mlp.fit(X_train, y_train)

#51% is baseline

CPU times: user 40min 19s, sys: 1min 47s, total: 42min 6s
Wall time: 25min 27s


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(140,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=400, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [27]:
def getAccuracy(model, X_train, X_test, y_train, y_test):
    y_pred_pre_train = model.predict(X_train)
    y_pred_train = np.array(y_pred_pre_train).astype(np.int)

    y_pred_pre_test = model.predict(X_test)
    y_pred_test = np.array(y_pred_pre_test).astype(np.int)

    num_same_train = 0
    num_same_test = 0

    for i in y_pred_train:
        if(y_pred_train[i] == y_train[i]):
            num_same_train += 1
    for i in y_pred_test:
        if(y_pred_test[i] == y_test[i]):
            num_same_test += 1

#    print(num_same_train)    
 #   print(num_same_test)

    accuracy_train = float(num_same_train) / y_train.size
    accuracy_test = float(num_same_test) / y_test.size
    
    return accuracy_train, accuracy_test

In [28]:
from sklearn.externals import joblib

import pickle
print("nn stats... ")

#y_train and y_test are ground truth
nn_acc_train, nn_acc_test = getAccuracy(mlp, X_train, X_test, y_train, y_test)
y_pred_pre = mlp.predict(X_test)
y_pred_test = np.array(y_pred_pre).astype(np.int)


nn_f1 = sklearn.metrics.f1_score(y_test, y_pred_test, average='macro')


print("F1 score: " +str(nn_f1))
print("training accuracy: " + str(nn_acc_train))
print("test accuracy: " + str(nn_acc_test))
if(nn_f1 > 0.37):
    print("Good model. Saving...")
    filename_acc = str(nn_acc_test)[2:4]
    f1 = str(nn_f1)[2:4]
    filename = "model_acc"+filename_acc+"hl"+str(num_nodes)+"f"+f1+".pkl"
 #   joblib.dump(mlp, filename)
    pickle.dump(mlp, open(filename, 'wb'))
    
    print("model saved")

#print("rf stats... ")
#rf_acc_train, rf_acc_test = getAccuracy(rf, X_train, X_test, y_train, y_test)
#print("training accuracy: " + str(rf_acc_train))
#print("test accuracy: " + str(rf_acc_test))

#100 hidden layer
#training accuracy: 0.870539291601
#test accuracy: 0.77532

#140 hidden layer
#training accuracy: 0.852538685422
#test accuracy: 0.32128

#100 2 hidden layers
#F1 = 48.761

nn stats... 
F1 score: 0.195525138436
training accuracy: 0.856217277759
test accuracy: 0.33478


In [21]:
from sklearn.externals import joblib


filename = "models/model_acc86hl140f65"
full_fn = filename+".pkl"

loaded_model = joblib.load(full_fn)
nn_acc_train, nn_acc_test = getAccuracy(loaded_model, X_train, X_test, y_train, y_test)
print("training accuracy: " + str(nn_acc_train))
print("test accuracy: " + str(nn_acc_test))

training accuracy: 0.541832570074
test accuracy: 0.86106


In [31]:
#loaded_model = mlp
import sklearn 

loaded_model = mlp
y_pred_pre = loaded_model.predict(X_test)
y_pred_test = np.array(y_pred_pre).astype(np.int)

print(y_pred_test)
print(y_pred_test.size)
f1 = sklearn.metrics.f1_score(y_test, y_pred_test, average='macro')
print(f1)
np.savetxt('english.output2.txt', y_pred_test, fmt='%i')




[ 7  7 11 ..., 10  9  0]
50000
0.195525138436


In [ ]:
n_classes = 20


# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y[:i], y_pred[:i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y.ravel(), y_pred.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()